In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
feature_x_sample_processed = pd.read_table(
    PATH_DICT["feature_x_sample.processed.tsv"], index_col=0
)

In [ ]:
for element_type in ("feature", "sample"):

    if element_type == "feature":

        element_alias = FEATURE_ALIAS

        df = feature_x_sample_processed

        elements_to_peek = FEATURES_TO_PEEK

    elif element_type == "sample":

        element_alias = SAMPLE_ALIAS

        df = feature_x_sample_processed.T

        elements_to_peek = SAMPLES_TO_PEEK

    element_x_fit_parameter = pd.read_table(
        PATH_DICT["{}_x_fit_parameter.tsv".format(element_type)], index_col=0
    )

    context_matrix = ccal.make_context_matrix(
        df,
        n_job=MAX_N_JOB,
        skew_t_pdf_fit_parameter=element_x_fit_parameter,
        output_file_path=PATH_DICT[
            "feature_x_sample.{}_context.tsv".format(element_type)
        ],
    )

    context_matrix_alias = "{} Context Matrix".format(element_alias)

    ccal.summarize_feature_x_sample(
        context_matrix,
        feature_x_sample_alias=context_matrix_alias,
        feature_alias=FEATURE_ALIAS,
        sample_alias=SAMPLE_ALIAS,
        feature_x_sample_value_name="{} Context".format(element_alias),
        plot_max_size=PLOT_MAX_SIZE,
    )

    for context in ("negative", "positive"):

        if context == "negative":

            element_context = context_matrix.clip(upper=0).sum(axis=1)

        elif context == "positive":

            element_context = context_matrix.clip(lower=0).sum(axis=1)

        element_context.sort_values(inplace=True)

        if elements_to_peek is None:

            ranks = []

        else:

            ranks = np.nonzero(
                [element in elements_to_peek for element in element_context.index]
            )[0]

        value_name = "{} {} Context Sum".format(element_alias, context.title())

        ccal.plot_points(
            (tuple(range(element_context.size)), ranks),
            (element_context, element_context[ranks]),
            names=("All", "Peek"),
            texts=(element_context.index, element_context.index[ranks]),
            modes=("markers", "markers+text"),
            title="{} Rank".format(value_name),
            xaxis_title="Rank",
            yaxis_title=value_name,
        )

        n_extreme = 3

        if context == "negative":

            ranks = range(n_extreme)

        elif context == "positive":

            ranks = range(-n_extreme, 0)

        for rank in ranks:

            ranked_element = element_context.index[rank]

            ranked_element_values = df.loc[ranked_element]

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                ranked_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            ccal.plot_context(
                ranked_element_values,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=ranked_element_values.dropna().size < PLOT_MAX_SIZE ** 0.5,
                title="{} Rank {}: {}".format(value_name, rank, ranked_element),
                xaxis_title=FEATURE_X_SAMPLE_VALUE_NAME,
            )